In [1]:
!pip install transformers datasets sentencepiece accelerate torch gradio -q

In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments, pipeline
from datasets import Dataset
import gradio as gr

In [3]:
data = [
    {"instruction": "Summarize the given text.",
     "input": "Machine learning is a field of artificial intelligence that enables systems to learn from data and improve automatically.",
     "output": "Machine learning enables systems to learn and improve from data."},
    {"instruction": "Explain what tokenization means in NLP.",
     "input": "",
     "output": "Tokenization is the process of splitting text into smaller units like words or subwords."},
    {"instruction": "Define embedding in simple terms.",
     "input": "",
     "output": "An embedding is a way to represent words as numerical vectors capturing their meaning."},
    {"instruction": "Summarize the text.",
     "input": "Artificial Intelligence enables machines to perform tasks that normally require human intelligence.",
     "output": "AI enables machines to mimic human intelligence."},
]

In [4]:
dataset = Dataset.from_list(data)

In [5]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
def preprocess(example):
    inp = f"Instruction: {example['instruction']} Input: {example['input']}"
    model_inputs = tokenizer(inp, truncation=True, padding="max_length", max_length=128)
    labels = tokenizer(example["output"], truncation=True, padding="max_length", max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_data = dataset.map(preprocess, batched=False)

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [7]:
model = T5ForConditionalGeneration.from_pretrained(model_name)
training_args = TrainingArguments(
    output_dir="./t5_instruction_finetune",
    learning_rate=3e-5,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_strategy="no"
)
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_data)
trainer.train()
trainer.save_model("./t5_instruction_model")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: arvindibram (arvindibram-abc7) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, mcp] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss


In [8]:
pipe = pipeline("text2text-generation", model="./t5_instruction_model", tokenizer=model_name)

Device set to use cuda:0


In [9]:
def generate_text(instruction, user_input):
    prompt = f"Instruction: {instruction} Input: {user_input}"
    result = pipe(prompt, max_length=80)[0]['generated_text']
    return result

demo = gr.Interface(
    fn=generate_text,
    inputs=[gr.Textbox(label="Instruction (e.g. Summarize the text)"),
            gr.Textbox(label="Input Text")],
    outputs=gr.Textbox(label="Generated Output"),
    title="🧠 Mini GenAI Instruction Model",
    description="Fine-tuned T5 model that follows your instructions to summarize or explain text."
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9d57b7aa488ef388dd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
